In [22]:
%load_ext autoreload
%aimport data_creater
%aimport model
%autoreload 1

from data_creater import *
from model import *
import re, copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()

In [4]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
symbol = 'AAPL'
# if os.path.isfile('./data/{0}/important_symbol.json'.format(symbol)):
#     with open('./data/{0}/important_symbol.json'.format(symbol), 'r') as json_file:
#         loaded_model_json = json_file.read()
#         loaded_model_json = json.loads(loaded_model_json)
#         symbols = list(loaded_model_json['score'].keys())

other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

box = []
for symbol_tem in other_symbols:
    box_tem = []
    for col in list(df.columns.values):
        if bool(re.match(symbol_tem + '_normal_.*', col)):
            box_tem.append(col)
    box.append(box_tem)

all_combination = []
all_combination.append(symbol_columns)
for box_tem in box:
    all_combination.append(symbol_columns + box_tem)
    
len(all_combination)

476

In [15]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn.svm import SVC

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings("ignore")

for i in range(len(all_combination)):
#     try:
    for window_size in [2,3,4,5,6]:
        print('{0}th combination, {1} window size'.format(i, window_size))
        ticker = 'AAPL'
        seq_obj = MultiSequence(ticker,window_size,1,'g001',all_combination[i])
        X_train,y_train,X_test,y_test = split_data(seq_obj)

        y_train = np.array([list(chain(*y_train.tolist()))]).tolist()[0]
        y_test = np.array([list(chain(*y_test.tolist()))]).tolist()[0]


        def bacon_F1_scoring(y_true, y_predict, verbose = False):
            table = pd.crosstab(np.array(y_true), y_predict, rownames = ['x'], colnames = ['y'])
            try:
                predict_1 = table.loc[:,1].tolist()
                precision = predict_1[1]/np.sum(predict_1)
                true_1 = table.loc[1,:].tolist()
                recall = true_1[1]/np.sum(true_1) 
                scoring = precision**10*recall
            except:
                scoring = 0

            if verbose is True:
                display(table)
            return scoring

        baconF1= make_scorer(bacon_F1_scoring, greater_is_better=True)

        # scores = ['precision', 'recall']

        param_grid = {'C': [0.1, 0.5, 1, 5, 10, 15, 20, 100, 150, 175, 200, 250, 1000],
                      'gamma': [0.01, 0.05, 0.1, 0.25, 0.5, 1, 2, 3, 4, 5, 10], }
        clf = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, scoring = baconF1)
        clf.fit(X_train, y_train)

        best_C=clf.best_estimator_.C
        best_gamma=clf.best_estimator_.gamma

        svm_rbf = svm.SVC(kernel='rbf', gamma=best_gamma, C=best_C)
        svm_rbf.fit(X_train, y_train)
        y_test_predict = svm_rbf.predict(X_test)
        score = bacon_F1_scoring(y_test, y_test_predict)
        print(score)
        print('-'*60)
#     except: 
#         pass


0th combination, 2 window size
0
------------------------------------------------------------
0th combination, 3 window size
2.384185791015625e-07
------------------------------------------------------------
0th combination, 4 window size
4.428674999999998e-06
------------------------------------------------------------
0th combination, 5 window size
0.008670764957916303
------------------------------------------------------------
0th combination, 6 window size
1.7074459273143301e-06
------------------------------------------------------------
1th combination, 2 window size
8.46754390421514e-06
------------------------------------------------------------
1th combination, 3 window size
2.384185791015625e-07
------------------------------------------------------------
1th combination, 4 window size
5.242880000000003e-05
------------------------------------------------------------
1th combination, 5 window size
4.124525003135204e-05
--------------------------------------------------------

2.384185791015625e-07
------------------------------------------------------------
16th combination, 3 window size
0.25
------------------------------------------------------------
16th combination, 4 window size
1.812548185416414e-06
------------------------------------------------------------
16th combination, 5 window size
0
------------------------------------------------------------
16th combination, 6 window size


KeyboardInterrupt: 

In [30]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
symbol = 'AAPL'
seq_obj = MultiSequence(symbol,2,1,'g001',all_combination[i])
X_train,y_train,X_test,y_test = split_data(seq_obj)
len(X_test)

68

In [17]:
window_sizes = [3,4,5]

In [30]:
result = model_selector('AAPL', window_sizes, target_length=1,target_theme='g0005',verbose=1,column=all_combination[3])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)


*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
1    > Window size: 3 Score: 0.5001177 , training score: 1.0000000 , testing score: 0.0002353training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.4545  testing_recall: 0.6250

Window size: 4
------------------------------------------------------------
2    > Window size: 4 Score: 0.5001221 , training score: 1.0000000 , testing score: 0.0002441training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.5000  testing_recall: 0.2500

Window size: 5
------------------------------------------------------------
3    > Window size: 5 Score: 0.5625000 , training score: 1.0000000 , testing score: 0.1250000training_precision: 1.0000  training_recall: 1.0000 testing_precision: 1.0000  testing_recall: 0.1250

Final model selection summary for AAPL with window size of 5. Score = 0.5625000, training score: 1.0000000 , testing score: 0.1250000
-----------

In [51]:
from data_creater import *
from model import *
import re, copy

stocks = companies()
symbols = stocks['Symbol'].values.tolist()
window_sizes = [5,6,7]
theme = 'g001'
symbol_chunk = 0

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

all_chunk_symbols = chunkIt(symbols, 5)
nowchunk_symbols = all_chunk_symbols[symbol_chunk]

for symbol in nowchunk_symbols:
#     try:
    symbol = 'XYL'
    other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
    df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
    symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

    box = []
    for symbol_tem in other_symbols:
        box_tem = []
        for col in list(df.columns.values):
            if bool(re.match(symbol_tem + '_normal_.*', col)):
                box_tem.append(col)
        box.append(box_tem)

    all_combination = []
    all_combination.append(symbol_columns)
    for box_tem in box:
        all_combination.append(symbol_columns + box_tem)

    for i in range(len(all_combination)):
        print(i)
        result = model_selector(symbol, window_sizes, target_length=1,target_theme=theme,verbose=1,column=all_combination[i])
        print("\nResults : ")
        print("-"*60)
        print(result[0])

        print(result[1])
        ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)
#     except:
#         with open('./rerun_symbol_model.txt', 'w') as f: 
#             f.write(symbol) 

0
*** Best Model Selection for XYL ***

Window size: 5
------------------------------------------------------------
1    > Window size: 5 Score: 0.0436384 , training score: 0.2615735 , testing score: 0.0000514 training_precision: 0.9600  training_recall: 0.3934 testing_precision: 0.5000  testing_recall: 0.0526

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1666689 , training score: 1.0000000 , testing score: 0.0000027 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.3333  testing_recall: 0.1579

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.2593567 , training score: 0.7666667 , testing score: 0.1578947 training_precision: 1.0000  training_recall: 0.7667 testing_precision: 1.0000  testing_recall: 0.1579

Final model selection summary for XYL with window size of 7. Score = 0.2593567, training score: 0.7666667 , testing score: 0.1578947
--------

1    > Window size: 5 Score: 0.0437158 , training score: 0.2622951 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 0.2623 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1681879 , training score: 1.0000000 , testing score: 0.0018254 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.6667  testing_recall: 0.1053

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.1681879 , training score: 1.0000000 , testing score: 0.0018254 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.6667  testing_recall: 0.1053

Final model selection summary for XYL with window size of 6. Score = 0.1681879, training score: 1.0000000 , testing score: 0.0018254
------------------------------------------------------------
training_precision: 1.0000  training_recall: 1.0000 testing_pre

1    > Window size: 5 Score: 0.1039689 , training score: 0.3606557 , testing score: 0.0526316 training_precision: 1.0000  training_recall: 0.3607 testing_precision: 1.0000  testing_recall: 0.0526

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1039689 , training score: 0.3606557 , testing score: 0.0526316 training_precision: 1.0000  training_recall: 0.3607 testing_precision: 1.0000  testing_recall: 0.0526

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.0944628 , training score: 0.5666667 , testing score: 0.0000221 training_precision: 1.0000  training_recall: 0.5667 testing_precision: 0.4000  testing_recall: 0.2105

Final model selection summary for XYL with window size of 5. Score = 0.1039689, training score: 0.3606557 , testing score: 0.0526316
------------------------------------------------------------
training_precision: 1.0000  training_recall: 0.5667 testing_pre

1    > Window size: 5 Score: 0.1530055 , training score: 0.9180328 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 0.9180 testing_precision: 0.2222  testing_recall: 0.1053

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1666667 , training score: 1.0000000 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.1666667 , training score: 1.0000000 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.0000  testing_recall: 0.0000

Final model selection summary for XYL with window size of 6. Score = 0.1666667, training score: 1.0000000 , testing score: 0.0000000
------------------------------------------------------------
training_precision: 1.0000  training_recall: 1.0000 testing_pre

1    > Window size: 5 Score: 0.0437158 , training score: 0.2622951 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 0.2623 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1639345 , training score: 0.9836066 , testing score: 0.0000001 training_precision: 1.0000  training_recall: 0.9836 testing_precision: 0.2353  testing_recall: 0.2105

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.0962986 , training score: 0.5333333 , testing score: 0.0088916 training_precision: 1.0000  training_recall: 0.5333 testing_precision: 0.7500  testing_recall: 0.1579

Final model selection summary for XYL with window size of 6. Score = 0.1639345, training score: 0.9836066 , testing score: 0.0000001
------------------------------------------------------------
training_precision: 1.0000  training_recall: 0.5333 testing_pre

...

1    > Window size: 5 Score: 0.1092896 , training score: 0.6557377 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 0.6557 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1393443 , training score: 0.8360656 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 0.8361 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.0972237 , training score: 0.5833333 , testing score: 0.0000018 training_precision: 1.0000  training_recall: 0.5833 testing_precision: 0.3333  testing_recall: 0.1053

Final model selection summary for XYL with window size of 6. Score = 0.1393443, training score: 0.8360656 , testing score: 0.0000000
------------------------------------------------------------
training_precision: 1.0000  training_recall: 0.5833 testing_pre

1    > Window size: 5 Score: 0.1557380 , training score: 0.9344262 , testing score: 0.0000004 training_precision: 1.0000  training_recall: 0.9344 testing_precision: 0.2667  testing_recall: 0.2105

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1281857 , training score: 0.7691126 , testing score: 0.0000004 training_precision: 0.9825  training_recall: 0.9180 testing_precision: 0.2727  testing_recall: 0.1579

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.1666680 , training score: 1.0000000 , testing score: 0.0000016 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.3077  testing_recall: 0.2105

Final model selection summary for XYL with window size of 7. Score = 0.1666680, training score: 1.0000000 , testing score: 0.0000016
------------------------------------------------------------
training_precision: 1.0000  training_recall: 1.0000 testing_pre

1    > Window size: 5 Score: 0.1612025 , training score: 0.9672131 , testing score: 0.0000004 training_precision: 1.0000  training_recall: 0.9672 testing_precision: 0.2857  testing_recall: 0.1053

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.0435956 , training score: 0.2615735 , testing score: 0.0000000 training_precision: 0.9600  training_recall: 0.3934 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.1668808 , training score: 1.0000000 , testing score: 0.0002570 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.5000  testing_recall: 0.2632

Final model selection summary for XYL with window size of 7. Score = 0.1668808, training score: 1.0000000 , testing score: 0.0002570
------------------------------------------------------------
training_precision: 1.0000  training_recall: 1.0000 testing_pre

1    > Window size: 5 Score: 0.1666667 , training score: 1.0000000 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 6
------------------------------------------------------------
2    > Window size: 6 Score: 0.1666667 , training score: 1.0000000 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.0000  testing_recall: 0.0000

Window size: 7
------------------------------------------------------------
3    > Window size: 7 Score: 0.1666667 , training score: 1.0000000 , testing score: 0.0000000 training_precision: 1.0000  training_recall: 1.0000 testing_precision: 0.0000  testing_recall: 0.0000

Final model selection summary for XYL with window size of 5. Score = 0.1666667, training score: 1.0000000 , testing score: 0.0000000
------------------------------------------------------------
training_precision: 1.0000  training_recall: 1.0000 testing_pre

KeyboardInterrupt: 

In [148]:
dropout =  0.25 ; rate = learn_rate = 0.001 ; epoch = 30
seq_obj = MultiSequence('AAPL',4,1,'s001', all_combination[1])
X_train,y_train,X_test,y_test = split_data(seq_obj)
y_train = list(chain(*y_train.tolist()))
y_test = list(chain(*y_test.tolist()))

from sklearn.svm import SVC
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train,y_train)
svm.predict(X_test)

# model = Sequential()
# model.add(Bidirectional(LSTM(X_train.shape[1],return_sequences=False),input_shape=(X_train.shape[1:])))
# model.add(Dense(X_train.shape[1]))
# model.add(Dropout(dropout))
# model.add(Dense(y_train.shape[1],activation='tanh'))
# optimizer = RMSprop(lr=learn_rate)
# model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
# model = final_model(X_train,y_train,rate,dropout)
# model.fit(X_train,y_train,epochs=epoch,batch_size=batch_size)

# raining_loss = model.evaluate(X_train,y_train,verbose=0)[0]
# training_acc = model.evaluate(X_train,y_train,verbose=0)[1]
# testing_loss = model.evaluate(X_test,y_test,verbose=0)[0]
# testing_acc = model.evaluate(X_test,y_test,verbose=0)[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [130]:
df.tail(60)

,date,close_price,ans,AAPL1_s001
186,2019-08-23,202.033447,-4.620%,0.238106
187,2019-08-26,205.871918,1.900%,0.436899
188,2019-08-27,203.548889,-1.130%,0.190585
189,2019-08-28,204.914795,0.670%,0.228366
190,2019-08-29,208.384369,1.690%,0.068299
191,2019-08-30,208.115189,-0.130%,0.197377
192,2019-09-03,205.084274,-1.460%,0.330177
193,2019-09-04,208.563843,1.700%,0.223813
194,2019-09-05,212.641586,1.960%,0.160933
195,2019-09-06,212.621643,-0.010%,0.192618


In [72]:
import os, re
symbol = 'AAPL'
path = './model_step/{0}'.format(symbol)
files = []
for r, d, f in os.walk(path):
    for file in f:
        if 'props' in file:
            files.append(os.path.join(r, file))

df = pd.DataFrame(columns=['other_symbol', 'score'])
for i in range(len(files)):
    try:
        with open(files[i], 'r') as json_file:
            loaded_model_json = json_file.read()
            loaded_model_json = json.loads(loaded_model_json)

        other_symbol_name = '.'.join(list((set([col.split('_')[0] for col in loaded_model_json['colname'] if not re.match('normal.*', col)]))))
        if other_symbol_name == '':
            other_symbol_name = symbol
        score = loaded_model_json['score']
        df.loc[i] = [other_symbol_name, score]
        
        if score > 0.7:
            print(files[i])
        
    except:
        continue
df = df.sort_values(by = 'score', ascending = False).head(30)
df.index = df['other_symbol']
df = df.drop(columns = ['other_symbol'])
df_json = df.to_dict()
print(df_json)
with open('./data/{0}/important_symbol.json'.format(symbol), 'w', encoding='utf-8') as f:
    json.dump(df_json, f, ensure_ascii=False, indent=4)

{'score': {'VIAB': 0.6628, 'FRC': 0.6616, 'PPL': 0.653, 'LOW': 0.6452, 'CAG': 0.6438, 'MKTX': 0.6429, 'ALGN': 0.6427, 'DLR': 0.6422, 'NRG': 0.6407, 'DFS': 0.6398, 'ABT': 0.6366, 'MRO': 0.6363, 'ROL': 0.6362, 'DISCK': 0.6352, 'MDLZ': 0.6342, 'CME': 0.6327, 'AVY': 0.6327, 'AMD': 0.6325, 'DXC': 0.6323, 'HRB': 0.6319, 'MOS': 0.6317, 'FANG': 0.6314, 'CERN': 0.6311, 'INCY': 0.6308, 'UPS': 0.6288, 'PNR': 0.6276, 'AXP': 0.6267, 'TFX': 0.6267, 'HSIC': 0.6262, 'FAST': 0.6252}}
